In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
font = {'size': 16}
matplotlib.rc('font', **font)
tqdm.tqdm.pandas()
import gc

In [2]:
%%time
training_data_df = pd.read_csv('./Prepped Data/training_data6_20220207.csv')
print(training_data_df.shape)
training_data_df.head()

(2091509, 18)
Wall time: 3.06 s


,주소_건축년도,건축년도,전용면적(㎡),층,x좌표,y좌표,계약날짜일수_2006년기준,계약날짜기준_건물연식,계약날짜일수_sin,계약날짜일수_cos,부동산유형,브랜드,대장구분명,지목,구,동,일괄계약,거래/보증금(만원)
0,서울특별시 강남구 개포동 12 1992,1992,33.18,1,206607.181143,443586.061143,2,14,0.034422,0.999407,0,0,0,0,0,0,0,15900
1,서울특별시 강남구 개포동 12 1992,1992,39.53,5,206607.181143,443586.061143,6,14,0.103102,0.994671,0,0,0,0,0,0,0,24800
2,서울특별시 강남구 개포동 12 1992,1992,39.53,12,206607.181143,443586.061143,8,14,0.137279,0.990532,0,0,0,0,0,0,0,24700
3,서울특별시 강남구 개포동 12 1992,1992,49.86,11,206607.181143,443586.061143,8,14,0.137279,0.990532,0,0,0,0,0,0,0,33300
4,서울특별시 강남구 개포동 12 1992,1992,39.53,4,206607.181143,443586.061143,9,14,0.154309,0.988023,0,0,0,0,0,0,0,24700


In [3]:
basedir = './Training/Training6_20220207/cat_col_maps/'

In [4]:
filenames = [f for f in os.listdir(basedir)]
filenames

['구.xlsx',
 '대장구분명.xlsx',
 '동.xlsx',
 '부동산유형.xlsx',
 '브랜드.xlsx',
 '일괄계약.xlsx',
 '지목.xlsx']

In [5]:
cat_cols = [col.replace('.xlsx', '') for col in filenames]
cat_cols

['구', '대장구분명', '동', '부동산유형', '브랜드', '일괄계약', '지목']

In [6]:
training_data_df = training_data_df.drop(columns=['대장구분명', '지목'])

In [7]:
training_data_df.head()

,주소_건축년도,건축년도,전용면적(㎡),층,x좌표,y좌표,계약날짜일수_2006년기준,계약날짜기준_건물연식,계약날짜일수_sin,계약날짜일수_cos,부동산유형,브랜드,구,동,일괄계약,거래/보증금(만원)
0,서울특별시 강남구 개포동 12 1992,1992,33.18,1,206607.181143,443586.061143,2,14,0.034422,0.999407,0,0,0,0,0,15900
1,서울특별시 강남구 개포동 12 1992,1992,39.53,5,206607.181143,443586.061143,6,14,0.103102,0.994671,0,0,0,0,0,24800
2,서울특별시 강남구 개포동 12 1992,1992,39.53,12,206607.181143,443586.061143,8,14,0.137279,0.990532,0,0,0,0,0,24700
3,서울특별시 강남구 개포동 12 1992,1992,49.86,11,206607.181143,443586.061143,8,14,0.137279,0.990532,0,0,0,0,0,33300
4,서울특별시 강남구 개포동 12 1992,1992,39.53,4,206607.181143,443586.061143,9,14,0.154309,0.988023,0,0,0,0,0,24700


In [8]:
cat_cols.remove('대장구분명')
cat_cols.remove('지목')

In [9]:
cat_cols

['구', '동', '부동산유형', '브랜드', '일괄계약']

# Training - Catboost Regressor

In [10]:
import catboost

In [11]:
from sklearn.model_selection import GroupKFold

In [12]:
gkfold = GroupKFold(n_splits=5)

In [13]:
#count = 0
models_list = []
for i, (train_idx, val_idx) in enumerate(gkfold.split(training_data_df, groups=training_data_df['주소_건축년도'])):
    print('-------------- Fold {} starting ---------------'.format(i))
    
    print(i)
    print(len(train_idx), len(val_idx))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]

    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]

    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)

    model = catboost.CatBoostRegressor(
        iterations=100000, depth=6, task_type='GPU', loss_function='RMSE'
    )

    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)

    model.save_model('./Training/Training6_20220207/catboost_RMSE_{}'.format(i))
    
    print('-------------- Fold {} ending ---------------'.format(i))
    #count += 1

-------------- Fold 0 starting ---------------
0
1673207 418302
Learning rate set to 0.037261
0:	learn: 36855.6118116	test: 38503.8746422	best: 38503.8746422 (0)	total: 59.3ms	remaining: 1h 38m 49s
50:	learn: 17761.0410813	test: 20610.9260723	best: 20610.9260723 (50)	total: 2.6s	remaining: 1h 24m 48s
100:	learn: 14416.2417705	test: 17516.9021616	best: 17516.9021616 (100)	total: 5.07s	remaining: 1h 23m 34s
150:	learn: 13189.1506155	test: 16526.3334738	best: 16526.3334738 (150)	total: 7.62s	remaining: 1h 23m 55s
200:	learn: 12444.5704730	test: 15888.0463343	best: 15888.0463343 (200)	total: 10.2s	remaining: 1h 24m 7s
250:	learn: 11912.2792429	test: 15491.6982950	best: 15491.6982950 (250)	total: 12.6s	remaining: 1h 23m 43s
300:	learn: 11495.4373858	test: 15235.2563891	best: 15229.8882881 (298)	total: 15.1s	remaining: 1h 23m 15s
350:	learn: 11163.5794520	test: 15039.7504440	best: 15039.7504440 (350)	total: 17.5s	remaining: 1h 22m 56s
400:	learn: 10925.8585879	test: 14888.5236733	best: 14888

900:	learn: 9759.1266967	test: 11609.5367826	best: 11609.2357846 (898)	total: 45.9s	remaining: 1h 24m 3s
950:	learn: 9665.5471896	test: 11557.4581299	best: 11557.4581299 (950)	total: 48.2s	remaining: 1h 23m 45s
1000:	learn: 9577.7767534	test: 11531.2593007	best: 11531.2593007 (1000)	total: 50.7s	remaining: 1h 23m 37s
1050:	learn: 9503.9574452	test: 11514.9450520	best: 11514.9450520 (1050)	total: 53.1s	remaining: 1h 23m 23s
1100:	learn: 9433.8260122	test: 11496.3702806	best: 11496.0920499 (1099)	total: 55.6s	remaining: 1h 23m 12s
1150:	learn: 9363.2592548	test: 11469.0060142	best: 11466.8577385 (1147)	total: 58s	remaining: 1h 23m 2s
1200:	learn: 9295.5146429	test: 11451.2240599	best: 11450.8514755 (1199)	total: 1m	remaining: 1h 22m 49s
1250:	learn: 9232.2602230	test: 11443.6092840	best: 11443.3696387 (1248)	total: 1m 2s	remaining: 1h 22m 42s
1300:	learn: 9175.2534084	test: 11429.0698995	best: 11429.0698995 (1300)	total: 1m 5s	remaining: 1h 22m 33s
1350:	learn: 9123.6255358	test: 11421.8

4700:	learn: 7394.2359196	test: 11051.8978944	best: 11051.8978944 (4700)	total: 3m 52s	remaining: 1h 18m 36s
4750:	learn: 7380.6096010	test: 11050.7401664	best: 11048.2269531 (4726)	total: 3m 55s	remaining: 1h 18m 34s
4800:	learn: 7365.6554146	test: 11044.6828001	best: 11043.8470875 (4775)	total: 3m 57s	remaining: 1h 18m 31s
4850:	learn: 7353.3081428	test: 11040.4077892	best: 11039.8278837 (4849)	total: 4m	remaining: 1h 18m 28s
4900:	learn: 7341.2296681	test: 11032.6971437	best: 11032.5026499 (4899)	total: 4m 2s	remaining: 1h 18m 24s
4950:	learn: 7329.8881500	test: 11028.9820820	best: 11027.4405140 (4945)	total: 4m 4s	remaining: 1h 18m 22s
5000:	learn: 7318.4523310	test: 11026.3811578	best: 11025.5245052 (4996)	total: 4m 7s	remaining: 1h 18m 19s
5050:	learn: 7303.5802423	test: 11025.0806930	best: 11024.9997499 (5046)	total: 4m 9s	remaining: 1h 18m 16s
5100:	learn: 7291.4307373	test: 11022.8704603	best: 11021.7037699 (5094)	total: 4m 12s	remaining: 1h 18m 13s
5150:	learn: 7279.9865017	t

3050:	learn: 7857.4701032	test: 13099.6955514	best: 13099.4506097 (3049)	total: 2m 31s	remaining: 1h 20m 13s
3100:	learn: 7836.9352549	test: 13097.1119509	best: 13097.1119509 (3100)	total: 2m 34s	remaining: 1h 20m 13s
3150:	learn: 7815.7890523	test: 13096.9565359	best: 13094.2322624 (3138)	total: 2m 36s	remaining: 1h 20m 16s
3200:	learn: 7794.1628866	test: 13101.9939535	best: 13090.0214636 (3191)	total: 2m 39s	remaining: 1h 20m 15s
3250:	learn: 7774.1034697	test: 13104.2651715	best: 13090.0214636 (3191)	total: 2m 41s	remaining: 1h 20m 12s
3300:	learn: 7754.8000055	test: 13101.6633390	best: 13090.0214636 (3191)	total: 2m 44s	remaining: 1h 20m 9s
3350:	learn: 7733.3468834	test: 13098.3904184	best: 13090.0214636 (3191)	total: 2m 46s	remaining: 1h 20m 5s
3400:	learn: 7711.6595275	test: 13088.4127617	best: 13087.9699509 (3397)	total: 2m 49s	remaining: 1h 20m 2s
3450:	learn: 7690.3289014	test: 13085.9495324	best: 13082.0112333 (3438)	total: 2m 51s	remaining: 1h 20m 1s
3500:	learn: 7671.36164

2750:	learn: 8084.1115001	test: 11376.6767645	best: 11376.6767645 (2750)	total: 2m 17s	remaining: 1h 21m 3s
2800:	learn: 8061.4660506	test: 11372.9612151	best: 11372.9612151 (2800)	total: 2m 20s	remaining: 1h 21m 1s
2850:	learn: 8037.7305021	test: 11354.5542104	best: 11351.7487614 (2841)	total: 2m 22s	remaining: 1h 20m 56s
2900:	learn: 8009.5461528	test: 11358.9100484	best: 11351.7487614 (2841)	total: 2m 24s	remaining: 1h 20m 51s
2950:	learn: 7986.3195416	test: 11347.1175691	best: 11346.7009165 (2949)	total: 2m 27s	remaining: 1h 20m 49s
3000:	learn: 7965.9673724	test: 11342.1047933	best: 11341.8457640 (2999)	total: 2m 29s	remaining: 1h 20m 46s
3050:	learn: 7942.1963967	test: 11328.6623777	best: 11327.6431416 (3043)	total: 2m 32s	remaining: 1h 20m 44s
3100:	learn: 7924.8544566	test: 11340.8091456	best: 11326.6804701 (3060)	total: 2m 34s	remaining: 1h 20m 42s
3150:	learn: 7902.4438431	test: 11336.0761875	best: 11326.6804701 (3060)	total: 2m 37s	remaining: 1h 20m 39s
3200:	learn: 7882.280

1600:	learn: 8877.6082891	test: 13151.7440606	best: 13141.9869511 (1599)	total: 1m 18s	remaining: 1h 20m 34s
1650:	learn: 8828.3672530	test: 13138.5691443	best: 13138.5691443 (1650)	total: 1m 21s	remaining: 1h 20m 37s
1700:	learn: 8782.9086601	test: 13128.7984246	best: 13125.6185864 (1671)	total: 1m 23s	remaining: 1h 20m 36s
1750:	learn: 8739.3203262	test: 13091.7605022	best: 13091.7605022 (1750)	total: 1m 26s	remaining: 1h 20m 37s
1800:	learn: 8696.3160330	test: 13086.9489904	best: 13085.1544664 (1799)	total: 1m 28s	remaining: 1h 20m 36s
1850:	learn: 8656.7245081	test: 13048.0341545	best: 13048.0341545 (1850)	total: 1m 31s	remaining: 1h 20m 36s
1900:	learn: 8615.1431828	test: 13038.3586260	best: 13037.8372075 (1896)	total: 1m 33s	remaining: 1h 20m 33s
1950:	learn: 8577.6171711	test: 13024.1298022	best: 13023.9157741 (1946)	total: 1m 36s	remaining: 1h 20m 31s
2000:	learn: 8540.3275139	test: 13008.9196664	best: 13008.9196664 (2000)	total: 1m 38s	remaining: 1h 20m 35s
2050:	learn: 8505.9

In [14]:
#count = 0
models_list = []
for i, (train_idx, val_idx) in enumerate(gkfold.split(training_data_df, groups=training_data_df['주소_건축년도'])):
    print('-------------- Fold {} starting ---------------'.format(i))
    
    print(i)
    print(len(train_idx), len(val_idx))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]

    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]

    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)

    model = catboost.CatBoostRegressor(
        iterations=100000, depth=6, task_type='GPU', loss_function='Quantile:alpha=0.5'
    )

    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)

    model.save_model('./Training/Training6_20220207/catboost_quantile05_{}'.format(i))
    
    print('-------------- Fold {} ending ---------------'.format(i))
    #count += 1

-------------- Fold 0 starting ---------------
0
1673207 418302
0:	learn: 10904.1418832	test: 11132.0617927	best: 11132.0617927 (0)	total: 106ms	remaining: 2h 56m 42s
50:	learn: 5742.7744636	test: 5942.7903476	best: 5942.7903476 (50)	total: 3.13s	remaining: 1h 42m 25s
100:	learn: 4528.8747274	test: 4709.5029333	best: 4709.5029333 (100)	total: 6.47s	remaining: 1h 46m 39s
150:	learn: 4037.3974816	test: 4277.2153707	best: 4277.2153707 (150)	total: 10s	remaining: 1h 50m 39s
200:	learn: 3777.1570977	test: 4057.9254606	best: 4057.9254606 (200)	total: 13.5s	remaining: 1h 51m 26s
250:	learn: 3612.1510082	test: 3926.8835243	best: 3926.8835243 (250)	total: 16.8s	remaining: 1h 50m 57s
300:	learn: 3490.5527409	test: 3827.3402087	best: 3827.3402087 (300)	total: 19.9s	remaining: 1h 49m 38s
350:	learn: 3383.5341114	test: 3736.0649865	best: 3736.0649865 (350)	total: 23s	remaining: 1h 48m 54s
400:	learn: 3294.2515015	test: 3674.1763797	best: 3674.1763797 (400)	total: 26.2s	remaining: 1h 48m 16s
450:	le

3850:	learn: 2225.7540161	test: 2990.4971623	best: 2990.4971623 (3850)	total: 4m 4s	remaining: 1h 41m 37s
3900:	learn: 2220.0150011	test: 2987.9206506	best: 2987.9206506 (3900)	total: 4m 7s	remaining: 1h 41m 33s
3950:	learn: 2215.4435261	test: 2984.9796941	best: 2984.9796941 (3950)	total: 4m 10s	remaining: 1h 41m 31s
4000:	learn: 2210.7035985	test: 2983.5773006	best: 2983.5519027 (3999)	total: 4m 13s	remaining: 1h 41m 28s
4050:	learn: 2205.6165149	test: 2981.8842081	best: 2981.8842081 (4050)	total: 4m 17s	remaining: 1h 41m 27s
4100:	learn: 2200.2860088	test: 2979.4071460	best: 2979.4071460 (4100)	total: 4m 20s	remaining: 1h 41m 24s
4150:	learn: 2195.5713262	test: 2976.7948324	best: 2976.7948324 (4150)	total: 4m 23s	remaining: 1h 41m 21s
4200:	learn: 2191.8177870	test: 2975.7945217	best: 2975.7945217 (4200)	total: 4m 26s	remaining: 1h 41m 17s
4250:	learn: 2187.4306765	test: 2973.3771294	best: 2973.3771294 (4250)	total: 4m 29s	remaining: 1h 41m 14s
4300:	learn: 2182.8053457	test: 2973.32

7700:	learn: 1971.1051555	test: 2907.8254084	best: 2907.5925432 (7694)	total: 8m 8s	remaining: 1h 37m 35s
7750:	learn: 1968.8836014	test: 2907.5781612	best: 2907.3602899 (7742)	total: 8m 11s	remaining: 1h 37m 31s
7800:	learn: 1966.7429840	test: 2908.2908329	best: 2907.3602899 (7742)	total: 8m 14s	remaining: 1h 37m 28s
7850:	learn: 1964.7252253	test: 2907.9631080	best: 2907.3602899 (7742)	total: 8m 17s	remaining: 1h 37m 23s
7900:	learn: 1962.7264385	test: 2907.8422384	best: 2907.3602899 (7742)	total: 8m 21s	remaining: 1h 37m 20s
7950:	learn: 1960.6699709	test: 2908.0129858	best: 2907.1580246 (7943)	total: 8m 24s	remaining: 1h 37m 16s
8000:	learn: 1958.4224068	test: 2907.7629846	best: 2907.1580246 (7943)	total: 8m 27s	remaining: 1h 37m 12s
8050:	learn: 1956.2225786	test: 2907.4530076	best: 2907.1580246 (7943)	total: 8m 30s	remaining: 1h 37m 8s
8100:	learn: 1953.8998916	test: 2907.2721622	best: 2906.9358693 (8064)	total: 8m 33s	remaining: 1h 37m 4s
8150:	learn: 1951.7718202	test: 2906.342

11500:	learn: 1835.8362211	test: 2881.2775459	best: 2877.8056811 (11380)	total: 12m 6s	remaining: 1h 33m 8s
11550:	learn: 1834.1278276	test: 2880.8941291	best: 2877.8056811 (11380)	total: 12m 9s	remaining: 1h 33m 4s
bestTest = 2877.805681
bestIteration = 11380
Shrink model to first 11381 iterations.
-------------- Fold 0 ending ---------------
-------------- Fold 1 starting ---------------
1
1673207 418302
0:	learn: 10853.4720880	test: 11344.8473878	best: 11344.8473878 (0)	total: 82.2ms	remaining: 2h 17m 1s
50:	learn: 5752.2775747	test: 6096.3541174	best: 6096.3541174 (50)	total: 2.98s	remaining: 1h 37m 30s
100:	learn: 4589.6002682	test: 4864.6799298	best: 4864.6799298 (100)	total: 6.17s	remaining: 1h 41m 47s
150:	learn: 4088.8016749	test: 4341.0844031	best: 4341.0844031 (150)	total: 9.51s	remaining: 1h 44m 51s
200:	learn: 3841.1926319	test: 4068.0289360	best: 4068.0289360 (200)	total: 12.8s	remaining: 1h 45m 39s
250:	learn: 3671.7128341	test: 3907.9476933	best: 3907.9476933 (250)	tota

3700:	learn: 2237.0443155	test: 2968.4603373	best: 2968.2969338 (3689)	total: 3m 47s	remaining: 1h 38m 30s
3750:	learn: 2232.0553046	test: 2967.9297350	best: 2967.9297350 (3750)	total: 3m 50s	remaining: 1h 38m 28s
3800:	learn: 2226.8658642	test: 2966.2657123	best: 2966.1436187 (3782)	total: 3m 53s	remaining: 1h 38m 25s
3850:	learn: 2221.5827880	test: 2964.1194352	best: 2964.1099493 (3849)	total: 3m 56s	remaining: 1h 38m 22s
3900:	learn: 2216.6242240	test: 2961.8366826	best: 2961.8118967 (3898)	total: 3m 59s	remaining: 1h 38m 18s
3950:	learn: 2211.5648333	test: 2960.5227228	best: 2960.4957949 (3948)	total: 4m 2s	remaining: 1h 38m 13s
4000:	learn: 2207.6768075	test: 2958.9437679	best: 2958.9437679 (4000)	total: 4m 5s	remaining: 1h 38m 10s
4050:	learn: 2202.4512592	test: 2958.3216719	best: 2958.3216719 (4050)	total: 4m 8s	remaining: 1h 38m 8s
4100:	learn: 2197.4311893	test: 2957.5435164	best: 2957.4902726 (4097)	total: 4m 11s	remaining: 1h 38m 7s
4150:	learn: 2193.3612470	test: 2955.85409

...

5500:	learn: 2091.6593536	test: 2782.5879675	best: 2782.5879675 (5500)	total: 5m 44s	remaining: 1h 38m 36s
5550:	learn: 2088.3542562	test: 2780.9113989	best: 2780.9113989 (5550)	total: 5m 47s	remaining: 1h 38m 33s
5600:	learn: 2085.0881738	test: 2779.6047831	best: 2779.5897892 (5591)	total: 5m 50s	remaining: 1h 38m 32s
5650:	learn: 2081.4975792	test: 2778.3608972	best: 2778.2489015 (5649)	total: 5m 53s	remaining: 1h 38m 29s
5700:	learn: 2077.9957244	test: 2776.7409384	best: 2776.6922845 (5694)	total: 5m 57s	remaining: 1h 38m 25s
5750:	learn: 2074.2372892	test: 2775.6283259	best: 2775.6283259 (5750)	total: 6m	remaining: 1h 38m 21s
5800:	learn: 2070.8373321	test: 2774.9554341	best: 2774.8324225 (5786)	total: 6m 3s	remaining: 1h 38m 17s
5850:	learn: 2066.7797828	test: 2772.6650315	best: 2772.6411636 (5838)	total: 6m 6s	remaining: 1h 38m 14s
5900:	learn: 2063.5658732	test: 2771.2002142	best: 2771.2002142 (5900)	total: 6m 9s	remaining: 1h 38m 11s
5950:	learn: 2060.2161000	test: 2770.1432936

2900:	learn: 2339.3730271	test: 3011.7832279	best: 3011.7832279 (2900)	total: 2m 58s	remaining: 1h 39m 30s
2950:	learn: 2333.1638481	test: 3008.8125823	best: 3008.8125823 (2950)	total: 3m 1s	remaining: 1h 39m 28s
3000:	learn: 2327.7638835	test: 3007.0338058	best: 3007.0338058 (3000)	total: 3m 4s	remaining: 1h 39m 26s
3050:	learn: 2320.0089505	test: 3004.4484139	best: 3004.0986562 (3042)	total: 3m 7s	remaining: 1h 39m 22s
3100:	learn: 2314.4350254	test: 3003.2335949	best: 3003.2335949 (3100)	total: 3m 10s	remaining: 1h 39m 19s
3150:	learn: 2307.5860025	test: 3000.6950210	best: 3000.4964272 (3146)	total: 3m 13s	remaining: 1h 39m 16s
3200:	learn: 2300.6918446	test: 2998.0551612	best: 2998.0551612 (3200)	total: 3m 16s	remaining: 1h 39m 12s
3250:	learn: 2294.4041769	test: 2996.3657366	best: 2996.2739367 (3248)	total: 3m 19s	remaining: 1h 39m 10s
3300:	learn: 2287.8476890	test: 2994.2932004	best: 2994.2858564 (3298)	total: 3m 23s	remaining: 1h 39m 7s
3350:	learn: 2281.9233951	test: 2992.9070

6750:	learn: 2022.2584449	test: 2921.9891322	best: 2921.9891322 (6750)	total: 6m 58s	remaining: 1h 36m 16s
6800:	learn: 2019.6540872	test: 2920.6849613	best: 2920.6849613 (6800)	total: 7m 1s	remaining: 1h 36m 12s
6850:	learn: 2016.5841521	test: 2920.0396078	best: 2919.9508679 (6837)	total: 7m 4s	remaining: 1h 36m 11s
6900:	learn: 2014.0181974	test: 2918.9569807	best: 2918.9110808 (6894)	total: 7m 7s	remaining: 1h 36m 8s
6950:	learn: 2011.7369747	test: 2918.8101009	best: 2918.8101009 (6950)	total: 7m 10s	remaining: 1h 36m 6s
7000:	learn: 2009.3957763	test: 2918.1864734	best: 2918.1864734 (7000)	total: 7m 13s	remaining: 1h 36m 5s
7050:	learn: 2006.5010256	test: 2918.2596073	best: 2917.8881236 (7022)	total: 7m 17s	remaining: 1h 36m 4s
7100:	learn: 2003.9082959	test: 2916.9067824	best: 2916.9012744 (7096)	total: 7m 20s	remaining: 1h 36m 2s
7150:	learn: 2001.5051327	test: 2916.0129572	best: 2915.9796032 (7148)	total: 7m 23s	remaining: 1h 35m 58s
7200:	learn: 1999.0164427	test: 2914.9499141	

10600:	learn: 1870.2141132	test: 2874.3054977	best: 2873.9220800 (10583)	total: 11m	remaining: 1h 32m 48s
10650:	learn: 1868.6432673	test: 2874.0588619	best: 2873.9220800 (10583)	total: 11m 3s	remaining: 1h 32m 46s
10700:	learn: 1867.1412711	test: 2873.5374384	best: 2873.4508404 (10694)	total: 11m 6s	remaining: 1h 32m 43s
10750:	learn: 1865.5190174	test: 2873.2871306	best: 2873.0741547 (10733)	total: 11m 9s	remaining: 1h 32m 40s
10800:	learn: 1864.0675110	test: 2872.2360214	best: 2872.2320434 (10799)	total: 11m 12s	remaining: 1h 32m 37s
10850:	learn: 1862.6721555	test: 2872.0692516	best: 2872.0640496 (10849)	total: 11m 16s	remaining: 1h 32m 34s
10900:	learn: 1861.3107659	test: 2871.8929957	best: 2871.8822857 (10899)	total: 11m 19s	remaining: 1h 32m 31s
10950:	learn: 1859.6147664	test: 2871.6188199	best: 2871.3966641 (10931)	total: 11m 22s	remaining: 1h 32m 28s
11000:	learn: 1858.2960636	test: 2871.1078004	best: 2871.1013744 (10999)	total: 11m 25s	remaining: 1h 32m 25s
11050:	learn: 185

14350:	learn: 1774.7395136	test: 2849.0978506	best: 2849.0978506 (14350)	total: 14m 56s	remaining: 1h 29m 10s
14400:	learn: 1773.9254032	test: 2848.7520709	best: 2848.7462569 (14399)	total: 14m 59s	remaining: 1h 29m 7s
14450:	learn: 1772.9145354	test: 2848.5944810	best: 2848.4938071 (14424)	total: 15m 2s	remaining: 1h 29m 3s
14500:	learn: 1771.8908157	test: 2848.0176715	best: 2848.0176715 (14500)	total: 15m 5s	remaining: 1h 29m
14550:	learn: 1771.0790003	test: 2847.9785035	best: 2847.8557976 (14531)	total: 15m 8s	remaining: 1h 28m 57s
14600:	learn: 1770.1173984	test: 2847.0751923	best: 2847.0546903 (14594)	total: 15m 12s	remaining: 1h 28m 55s
14650:	learn: 1769.1164757	test: 2847.0488763	best: 2847.0488763 (14650)	total: 15m 15s	remaining: 1h 28m 52s
14700:	learn: 1767.9642364	test: 2847.2266621	best: 2846.9766603 (14695)	total: 15m 18s	remaining: 1h 28m 49s
14750:	learn: 1766.7073454	test: 2846.7373685	best: 2846.6403666 (14745)	total: 15m 21s	remaining: 1h 28m 46s
14800:	learn: 1765.

In [15]:
count = 0
models_list = []
for train_idx, val_idx in gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']):
    print('-------------- Fold {} starting ---------------'.format(count))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]
    
    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]
    
    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)
    
    model = catboost.CatBoostRegressor(iterations=10000, depth=6, task_type='GPU', loss_function='Quantile:alpha=0.25')
    
    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)
    
    model.save_model('./Training/Training6_20220207/catboost_quantile025_{}'.format(count))
    
    print('-------------- Fold {} ending ---------------'.format(count))
    count += 1

-------------- Fold 0 starting ---------------
0:	learn: 6802.1755109	test: 6935.6867718	best: 6935.6867718 (0)	total: 99.9ms	remaining: 16m 39s
50:	learn: 4255.1351889	test: 4381.5741928	best: 4381.5741928 (50)	total: 3.06s	remaining: 9m 57s
100:	learn: 3459.4668944	test: 3569.6370182	best: 3569.6370182 (100)	total: 6.19s	remaining: 10m 6s
150:	learn: 3074.6796350	test: 3208.6049983	best: 3208.6049983 (150)	total: 9.51s	remaining: 10m 20s
200:	learn: 2868.4493335	test: 3018.2442350	best: 3018.2442350 (200)	total: 12.8s	remaining: 10m 26s
250:	learn: 2748.8672950	test: 2915.8456426	best: 2915.8456426 (250)	total: 16.2s	remaining: 10m 30s
300:	learn: 2661.4417941	test: 2843.9924839	best: 2843.9924839 (300)	total: 19.5s	remaining: 10m 29s
350:	learn: 2586.9970852	test: 2792.3572921	best: 2792.3572921 (350)	total: 22.8s	remaining: 10m 26s
400:	learn: 2527.9325081	test: 2744.4666102	best: 2744.4666102 (400)	total: 25.9s	remaining: 10m 20s
450:	learn: 2479.0725451	test: 2706.5521083	best: 2

4000:	learn: 1714.5924539	test: 2244.1805585	best: 2244.0685629 (3981)	total: 4m 8s	remaining: 6m 13s
4050:	learn: 1711.5627558	test: 2243.5980894	best: 2243.5948764 (4049)	total: 4m 12s	remaining: 6m 10s
4100:	learn: 1707.8592475	test: 2244.2015195	best: 2242.3835793 (4086)	total: 4m 15s	remaining: 6m 7s
4150:	learn: 1703.9577578	test: 2245.1302265	best: 2242.3835793 (4086)	total: 4m 18s	remaining: 6m 4s
4200:	learn: 1700.6301695	test: 2245.0881516	best: 2242.3835793 (4086)	total: 4m 21s	remaining: 6m 1s
4250:	learn: 1697.0778248	test: 2243.8109117	best: 2242.3835793 (4086)	total: 4m 24s	remaining: 5m 57s
bestTest = 2242.383579
bestIteration = 4086
Shrink model to first 4087 iterations.
-------------- Fold 0 ending ---------------
-------------- Fold 1 starting ---------------
0:	learn: 6773.6233377	test: 7045.7625735	best: 7045.7625735 (0)	total: 115ms	remaining: 19m 8s
50:	learn: 4252.3509022	test: 4449.3633786	best: 4449.3633786 (50)	total: 3.1s	remaining: 10m 5s
100:	learn: 3443.4

3650:	learn: 1762.7330127	test: 2232.8058102	best: 2232.8058102 (3650)	total: 3m 46s	remaining: 6m 33s
3700:	learn: 1758.9257707	test: 2231.1480605	best: 2231.1480605 (3700)	total: 3m 49s	remaining: 6m 30s
3750:	learn: 1753.9806707	test: 2229.5351397	best: 2229.5351397 (3750)	total: 3m 52s	remaining: 6m 27s
3800:	learn: 1749.8074536	test: 2227.0571597	best: 2227.0571597 (3800)	total: 3m 55s	remaining: 6m 24s
3850:	learn: 1745.6421925	test: 2225.4577028	best: 2225.4577028 (3850)	total: 3m 58s	remaining: 6m 20s
3900:	learn: 1741.4711174	test: 2223.6852035	best: 2223.6852035 (3900)	total: 4m 1s	remaining: 6m 17s
3950:	learn: 1737.0766630	test: 2221.6206282	best: 2221.6206282 (3950)	total: 4m 4s	remaining: 6m 14s
4000:	learn: 1733.2221441	test: 2219.7901421	best: 2219.5794617 (3994)	total: 4m 7s	remaining: 6m 11s
4050:	learn: 1729.8188210	test: 2218.7122605	best: 2218.6730926 (4041)	total: 4m 10s	remaining: 6m 8s
4100:	learn: 1726.0143330	test: 2216.9945350	best: 2216.9945350 (4100)	total:

7650:	learn: 1551.7832856	test: 2150.7843042	best: 2150.3915544 (7632)	total: 7m 55s	remaining: 2m 26s
7700:	learn: 1550.0745861	test: 2151.1697099	best: 2150.3915544 (7632)	total: 7m 59s	remaining: 2m 23s
7750:	learn: 1548.2615409	test: 2149.7073406	best: 2149.6400208 (7743)	total: 8m 2s	remaining: 2m 19s
7800:	learn: 1546.6490781	test: 2148.9749320	best: 2148.9749320 (7800)	total: 8m 5s	remaining: 2m 16s
7850:	learn: 1544.9357886	test: 2148.1917275	best: 2148.1917275 (7850)	total: 8m 8s	remaining: 2m 13s
7900:	learn: 1543.1409503	test: 2147.5971714	best: 2147.5867675 (7898)	total: 8m 11s	remaining: 2m 10s
7950:	learn: 1541.7249390	test: 2147.7116151	best: 2147.2379286 (7930)	total: 8m 14s	remaining: 2m 7s
8000:	learn: 1540.1248692	test: 2146.7059493	best: 2146.7059493 (8000)	total: 8m 17s	remaining: 2m 4s
8050:	learn: 1538.5628963	test: 2146.0959403	best: 2146.0869133 (8048)	total: 8m 20s	remaining: 2m 1s
8100:	learn: 1537.0534022	test: 2145.5818619	best: 2145.5093401 (8095)	total: 8

1600:	learn: 1986.4702933	test: 2459.2383493	best: 2459.2383493 (1600)	total: 1m 40s	remaining: 8m 47s
1650:	learn: 1977.3542712	test: 2453.2661665	best: 2453.2661665 (1650)	total: 1m 43s	remaining: 8m 43s
1700:	learn: 1968.9825921	test: 2448.0678553	best: 2448.0678553 (1700)	total: 1m 46s	remaining: 8m 40s
1750:	learn: 1959.5359379	test: 2444.7532740	best: 2444.7532740 (1750)	total: 1m 49s	remaining: 8m 37s
1800:	learn: 1949.0851473	test: 2439.1527270	best: 2439.1527270 (1800)	total: 1m 52s	remaining: 8m 33s
1850:	learn: 1940.2595184	test: 2434.9290226	best: 2434.9290226 (1850)	total: 1m 55s	remaining: 8m 30s
1900:	learn: 1931.4205786	test: 2430.4383340	best: 2430.4383340 (1900)	total: 1m 59s	remaining: 8m 27s
1950:	learn: 1923.6322870	test: 2426.3728311	best: 2426.3728311 (1950)	total: 2m 2s	remaining: 8m 24s
2000:	learn: 1917.0891061	test: 2423.1295476	best: 2423.1295476 (2000)	total: 2m 5s	remaining: 8m 21s
2050:	learn: 1909.9443309	test: 2418.4800455	best: 2418.4800455 (2050)	tota

...

3500:	learn: 1770.3663157	test: 2175.7212349	best: 2175.7212349 (3500)	total: 3m 36s	remaining: 6m 41s
3550:	learn: 1765.9726262	test: 2174.0666982	best: 2174.0666982 (3550)	total: 3m 39s	remaining: 6m 38s
3600:	learn: 1762.6765559	test: 2173.1549742	best: 2173.1549742 (3600)	total: 3m 42s	remaining: 6m 35s
3650:	learn: 1758.3223403	test: 2171.7666566	best: 2171.7652796 (3649)	total: 3m 45s	remaining: 6m 32s
3700:	learn: 1754.6342849	test: 2170.8914994	best: 2170.8104097 (3698)	total: 3m 48s	remaining: 6m 29s
3750:	learn: 1750.8434139	test: 2169.4909420	best: 2169.4909420 (3750)	total: 3m 52s	remaining: 6m 26s
3800:	learn: 1746.2500599	test: 2168.2689349	best: 2168.1046134 (3796)	total: 3m 55s	remaining: 6m 23s
3850:	learn: 1742.9015107	test: 2168.2586839	best: 2167.4557425 (3837)	total: 3m 58s	remaining: 6m 20s
3900:	learn: 1738.8869542	test: 2167.0483048	best: 2167.0483048 (3900)	total: 4m 1s	remaining: 6m 17s
3950:	learn: 1735.0295283	test: 2164.9139617	best: 2164.6148476 (3944)	tot

7500:	learn: 1552.5531796	test: 2102.2510244	best: 2102.1381107 (7498)	total: 7m 46s	remaining: 2m 35s
7550:	learn: 1550.8532011	test: 2102.3132952	best: 2102.0808889 (7512)	total: 7m 49s	remaining: 2m 32s
7600:	learn: 1549.1633205	test: 2101.8313467	best: 2101.8214018 (7573)	total: 7m 52s	remaining: 2m 29s
7650:	learn: 1547.5220938	test: 2101.0285583	best: 2100.9748555 (7648)	total: 7m 56s	remaining: 2m 26s
7700:	learn: 1545.8738291	test: 2100.6452945	best: 2100.4684271 (7679)	total: 7m 59s	remaining: 2m 23s
7750:	learn: 1543.9411382	test: 2100.3659174	best: 2100.2375509 (7741)	total: 8m 2s	remaining: 2m 19s
7800:	learn: 1542.1369669	test: 2099.9517478	best: 2099.9329288 (7788)	total: 8m 5s	remaining: 2m 16s
7850:	learn: 1540.5531151	test: 2099.2403001	best: 2099.2191861 (7843)	total: 8m 8s	remaining: 2m 13s
7900:	learn: 1538.7689867	test: 2098.6530880	best: 2098.6281490 (7899)	total: 8m 11s	remaining: 2m 10s
7950:	learn: 1536.9955684	test: 2098.6462030	best: 2098.6064231 (7912)	total

1400:	learn: 2084.6101812	test: 2358.5060519	best: 2358.5060519 (1400)	total: 1m 27s	remaining: 8m 55s
1450:	learn: 2070.6708550	test: 2350.9986541	best: 2350.9986541 (1450)	total: 1m 30s	remaining: 8m 52s
1500:	learn: 2059.2084379	test: 2346.1344821	best: 2346.1344821 (1500)	total: 1m 33s	remaining: 8m 48s
1550:	learn: 2048.2236853	test: 2341.1901382	best: 2341.1901382 (1550)	total: 1m 36s	remaining: 8m 45s
1600:	learn: 2034.9794359	test: 2336.1631744	best: 2336.1631744 (1600)	total: 1m 39s	remaining: 8m 42s
1650:	learn: 2023.5736286	test: 2332.3202765	best: 2332.3202765 (1650)	total: 1m 42s	remaining: 8m 38s
1700:	learn: 2009.9245426	test: 2326.9940450	best: 2326.9940450 (1700)	total: 1m 45s	remaining: 8m 35s
1750:	learn: 1998.6668388	test: 2322.1698060	best: 2322.1698060 (1750)	total: 1m 48s	remaining: 8m 32s
1800:	learn: 1988.5518883	test: 2318.6336729	best: 2318.6336729 (1800)	total: 1m 51s	remaining: 8m 29s
1850:	learn: 1978.4459649	test: 2313.4400922	best: 2313.4134702 (1849)	to

5400:	learn: 1647.6036548	test: 2200.6335916	best: 2200.6335916 (5400)	total: 5m 37s	remaining: 4m 47s
5450:	learn: 1645.1019598	test: 2200.0544871	best: 2199.9082192 (5445)	total: 5m 40s	remaining: 4m 44s
5500:	learn: 1642.7137905	test: 2199.5769745	best: 2199.4287176 (5491)	total: 5m 43s	remaining: 4m 40s
5550:	learn: 1640.2322915	test: 2198.9244300	best: 2198.9213700 (5549)	total: 5m 46s	remaining: 4m 37s
5600:	learn: 1637.8344832	test: 2197.9131009	best: 2197.9131009 (5600)	total: 5m 49s	remaining: 4m 34s
5650:	learn: 1635.3315643	test: 2197.1666145	best: 2197.1666145 (5650)	total: 5m 52s	remaining: 4m 31s
5700:	learn: 1632.4156997	test: 2196.2045513	best: 2196.0896484 (5686)	total: 5m 55s	remaining: 4m 28s
5750:	learn: 1629.7088324	test: 2194.7129555	best: 2194.7129555 (5750)	total: 5m 59s	remaining: 4m 25s
5800:	learn: 1627.2053014	test: 2193.9641741	best: 2193.9635621 (5799)	total: 6m 2s	remaining: 4m 22s
5850:	learn: 1625.0425004	test: 2193.6283394	best: 2193.6283394 (5850)	tot

9400:	learn: 1497.7147802	test: 2156.6193530	best: 2156.5003191 (9380)	total: 9m 48s	remaining: 37.5s
9450:	learn: 1496.4845112	test: 2156.4738502	best: 2156.3532863 (9422)	total: 9m 51s	remaining: 34.4s
9500:	learn: 1495.1263346	test: 2156.1000715	best: 2156.0892085 (9498)	total: 9m 54s	remaining: 31.2s
9550:	learn: 1493.7586720	test: 2156.3047853	best: 2156.0867605 (9501)	total: 9m 57s	remaining: 28.1s
9600:	learn: 1492.2750357	test: 2155.9510496	best: 2155.9496726 (9599)	total: 10m 1s	remaining: 25s
9650:	learn: 1490.9450110	test: 2155.6248539	best: 2155.6248539 (9650)	total: 10m 4s	remaining: 21.8s
9700:	learn: 1489.5481255	test: 2155.3280341	best: 2155.3280341 (9700)	total: 10m 7s	remaining: 18.7s
9750:	learn: 1488.2358487	test: 2155.0500333	best: 2155.0500333 (9750)	total: 10m 10s	remaining: 15.6s
9800:	learn: 1487.0016017	test: 2154.1647761	best: 2154.1647761 (9800)	total: 10m 13s	remaining: 12.5s
9850:	learn: 1485.8987813	test: 2153.7420374	best: 2153.7365294 (9849)	total: 10m 

In [16]:
count = 0
models_list = []
for train_idx, val_idx in gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']):
    print('-------------- Fold {} starting ---------------'.format(count))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]
    
    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]
    
    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)
    
    model = catboost.CatBoostRegressor(iterations=10000, depth=6, task_type='GPU', loss_function='Quantile:alpha=0.75')
    
    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)
    
    model.save_model('./Training/Training6_20220207/catboost_quantile075_{}'.format(count))
    
    print('-------------- Fold {} ending ---------------'.format(count))
    count += 1

-------------- Fold 0 starting ---------------
0:	learn: 11539.1049069	test: 11824.9463785	best: 11824.9463785 (0)	total: 81.3ms	remaining: 13m 32s
50:	learn: 4950.3145708	test: 5288.2327505	best: 5288.2327505 (50)	total: 3.14s	remaining: 10m 12s
100:	learn: 3759.6511896	test: 4122.9438253	best: 4122.9438253 (100)	total: 6.23s	remaining: 10m 10s
150:	learn: 3391.3208730	test: 3773.7328533	best: 3773.7328533 (150)	total: 9.45s	remaining: 10m 16s
200:	learn: 3183.9614345	test: 3596.7160951	best: 3596.7160951 (200)	total: 12.7s	remaining: 10m 17s
250:	learn: 3060.4522023	test: 3487.3110432	best: 3487.3110432 (250)	total: 15.8s	remaining: 10m 13s
300:	learn: 2958.9290124	test: 3399.1781249	best: 3399.1781249 (300)	total: 18.8s	remaining: 10m 6s
350:	learn: 2875.7009025	test: 3330.2665347	best: 3330.2665347 (350)	total: 21.9s	remaining: 10m
400:	learn: 2807.4600214	test: 3274.0055558	best: 3274.0055558 (400)	total: 24.9s	remaining: 9m 56s
450:	learn: 2744.0377096	test: 3231.3055352	best: 32

4000:	learn: 1831.2938160	test: 2694.1713881	best: 2694.1355863 (3997)	total: 4m 6s	remaining: 6m 8s
4050:	learn: 1826.8317357	test: 2691.1579098	best: 2691.1579098 (4050)	total: 4m 9s	remaining: 6m 5s
4100:	learn: 1822.8936790	test: 2688.4868444	best: 2688.4868444 (4100)	total: 4m 12s	remaining: 6m 2s
4150:	learn: 1819.4758210	test: 2686.2756956	best: 2686.2144957 (4149)	total: 4m 15s	remaining: 5m 59s
4200:	learn: 1816.3858411	test: 2685.1318712	best: 2685.1318712 (4200)	total: 4m 18s	remaining: 5m 56s
4250:	learn: 1812.7504176	test: 2683.3203571	best: 2683.3203571 (4250)	total: 4m 21s	remaining: 5m 53s
4300:	learn: 1808.9797425	test: 2681.7701661	best: 2681.1902979 (4285)	total: 4m 24s	remaining: 5m 50s
4350:	learn: 1804.5641741	test: 2680.3163647	best: 2680.3163647 (4350)	total: 4m 27s	remaining: 5m 47s
4400:	learn: 1800.3175172	test: 2677.3469503	best: 2677.3374643 (4397)	total: 4m 30s	remaining: 5m 44s
4450:	learn: 1796.6181399	test: 2676.8549039	best: 2676.0853164 (4442)	total: 

950:	learn: 2392.6416947	test: 2773.4168711	best: 2773.3434313 (949)	total: 57.7s	remaining: 9m 9s
1000:	learn: 2367.5315535	test: 2762.2720809	best: 2762.2720809 (1000)	total: 1m	remaining: 9m 5s
1050:	learn: 2341.9983684	test: 2748.6612447	best: 2748.6612447 (1050)	total: 1m 3s	remaining: 9m 2s
1100:	learn: 2317.6506242	test: 2734.9301509	best: 2734.9301509 (1100)	total: 1m 6s	remaining: 8m 59s
1150:	learn: 2300.1369131	test: 2724.6274701	best: 2724.6274701 (1150)	total: 1m 9s	remaining: 8m 56s
1200:	learn: 2278.7922451	test: 2716.0992776	best: 2715.4496416 (1193)	total: 1m 12s	remaining: 8m 52s
1250:	learn: 2261.0840715	test: 2708.5567078	best: 2708.4288002 (1248)	total: 1m 15s	remaining: 8m 49s
1300:	learn: 2247.2789177	test: 2703.2586409	best: 2703.2586409 (1300)	total: 1m 18s	remaining: 8m 46s
1350:	learn: 2228.3846912	test: 2700.3938781	best: 2700.1894708 (1346)	total: 1m 21s	remaining: 8m 43s
1400:	learn: 2213.3411586	test: 2694.9966675	best: 2694.1921961 (1397)	total: 1m 24s	r

800:	learn: 2417.3711728	test: 3245.8306965	best: 3245.8306965 (800)	total: 49.3s	remaining: 9m 25s
850:	learn: 2389.0177462	test: 3234.1198082	best: 3234.1198082 (850)	total: 52.3s	remaining: 9m 22s
900:	learn: 2363.9054630	test: 3220.6212736	best: 3220.6212736 (900)	total: 55.3s	remaining: 9m 18s
950:	learn: 2342.8462922	test: 3208.6429422	best: 3208.5609344 (949)	total: 58.4s	remaining: 9m 15s
1000:	learn: 2316.0571334	test: 3193.6287945	best: 3193.6287945 (1000)	total: 1m 1s	remaining: 9m 11s
1050:	learn: 2287.0237765	test: 3185.0130862	best: 3185.0130862 (1050)	total: 1m 4s	remaining: 9m 8s
1100:	learn: 2269.2822490	test: 3178.7633624	best: 3178.6464707 (1098)	total: 1m 7s	remaining: 9m 4s
1150:	learn: 2253.7246378	test: 3173.2660900	best: 3173.2660900 (1150)	total: 1m 10s	remaining: 9m 1s
1200:	learn: 2238.9785364	test: 3164.0408317	best: 3164.0408317 (1200)	total: 1m 13s	remaining: 8m 58s
1250:	learn: 2222.8944536	test: 3157.6806422	best: 3157.6806422 (1250)	total: 1m 16s	remain

500:	learn: 2713.4387078	test: 2987.1372932	best: 2987.1372932 (500)	total: 30.5s	remaining: 9m 37s
550:	learn: 2658.1789247	test: 2950.7059684	best: 2950.7059684 (550)	total: 33.5s	remaining: 9m 33s
600:	learn: 2607.4892395	test: 2921.4521566	best: 2921.4521566 (600)	total: 36.4s	remaining: 9m 29s
650:	learn: 2563.3994981	test: 2885.9458669	best: 2885.9458669 (650)	total: 39.5s	remaining: 9m 26s
700:	learn: 2523.5409606	test: 2857.9463450	best: 2857.9463450 (700)	total: 42.4s	remaining: 9m 22s
750:	learn: 2489.3299442	test: 2832.1460380	best: 2832.1460380 (750)	total: 45.5s	remaining: 9m 19s
800:	learn: 2460.0693829	test: 2812.1970442	best: 2812.1970442 (800)	total: 48.5s	remaining: 9m 16s
850:	learn: 2434.5502738	test: 2796.6146564	best: 2796.6146564 (850)	total: 51.5s	remaining: 9m 13s
900:	learn: 2411.4468754	test: 2780.7807374	best: 2780.7807374 (900)	total: 54.4s	remaining: 9m 9s
950:	learn: 2379.4874011	test: 2756.5997772	best: 2756.5997772 (950)	total: 57.5s	remaining: 9m 7s
10

4550:	learn: 1792.2073145	test: 2440.0021802	best: 2439.9712743 (4549)	total: 4m 36s	remaining: 5m 31s
4600:	learn: 1788.6171789	test: 2440.8559175	best: 2439.6637453 (4562)	total: 4m 39s	remaining: 5m 28s
4650:	learn: 1785.1570929	test: 2439.5831146	best: 2439.5831146 (4650)	total: 4m 42s	remaining: 5m 25s
4700:	learn: 1781.4067668	test: 2437.2608881	best: 2437.2608881 (4700)	total: 4m 45s	remaining: 5m 22s
4750:	learn: 1777.2737886	test: 2434.5345898	best: 2434.5345898 (4750)	total: 4m 48s	remaining: 5m 19s
4800:	learn: 1774.4550770	test: 2433.1045034	best: 2433.0945585 (4799)	total: 4m 51s	remaining: 5m 16s
4850:	learn: 1771.1297837	test: 2432.1787034	best: 2431.9534881 (4848)	total: 4m 54s	remaining: 5m 13s
4900:	learn: 1767.9871719	test: 2432.2224613	best: 2431.9534881 (4848)	total: 4m 57s	remaining: 5m 9s
4950:	learn: 1764.6314317	test: 2432.4074377	best: 2431.9534881 (4848)	total: 5m	remaining: 5m 6s
5000:	learn: 1761.4009982	test: 2431.7593318	best: 2431.5535474 (4986)	total: 5

1800:	learn: 2124.5725815	test: 2786.9875520	best: 2786.5536444 (1799)	total: 1m 48s	remaining: 8m 14s
1850:	learn: 2112.6914048	test: 2782.7981095	best: 2782.7001896 (1845)	total: 1m 51s	remaining: 8m 11s
1900:	learn: 2101.1627389	test: 2777.8837536	best: 2777.8837536 (1900)	total: 1m 54s	remaining: 8m 8s
1950:	learn: 2092.8062118	test: 2776.0471431	best: 2775.6701514 (1946)	total: 1m 57s	remaining: 8m 5s
2000:	learn: 2083.7146750	test: 2774.4691024	best: 2774.1276067 (1994)	total: 2m	remaining: 8m 1s
2050:	learn: 2074.4761058	test: 2771.9739805	best: 2771.6058628 (2046)	total: 2m 3s	remaining: 7m 58s
2100:	learn: 2064.8048539	test: 2768.0155677	best: 2768.0155677 (2100)	total: 2m 6s	remaining: 7m 55s
2150:	learn: 2057.2593772	test: 2766.4084571	best: 2766.2676972 (2149)	total: 2m 9s	remaining: 7m 52s
2200:	learn: 2049.3075338	test: 2764.1091750	best: 2764.1091750 (2200)	total: 2m 12s	remaining: 7m 49s
2250:	learn: 2042.7954158	test: 2763.0644918	best: 2762.9084320 (2234)	total: 2m 15

In [17]:
count = 0
models_list = []
for train_idx, val_idx in gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']):
    print('-------------- Fold {} starting ---------------'.format(count))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]
    
    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]
    
    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)
    
    model = catboost.CatBoostRegressor(iterations=10000, depth=6, task_type='GPU', loss_function='Quantile:alpha=0.125')
    
    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)
    
    model.save_model('./Training/Training6_20220207/catboost_quantile0125_{}'.format(count))
    
    print('-------------- Fold {} ending ---------------'.format(count))
    count += 1

-------------- Fold 0 starting ---------------
0:	learn: 3771.9030915	test: 3838.4342030	best: 3838.4342030 (0)	total: 110ms	remaining: 18m 20s
50:	learn: 2612.6805159	test: 2676.0170786	best: 2676.0170786 (50)	total: 3.03s	remaining: 9m 51s
100:	learn: 2148.3663599	test: 2208.6897218	best: 2208.6897218 (100)	total: 5.98s	remaining: 9m 45s
150:	learn: 1934.7463308	test: 2000.2260568	best: 2000.2260568 (150)	total: 9.09s	remaining: 9m 52s
200:	learn: 1812.2759659	test: 1892.8399864	best: 1892.8399864 (200)	total: 12.4s	remaining: 10m 2s
250:	learn: 1724.9646027	test: 1817.8644137	best: 1817.8644137 (250)	total: 15.8s	remaining: 10m 12s
300:	learn: 1672.2611368	test: 1773.4960866	best: 1773.4960866 (300)	total: 19s	remaining: 10m 11s
350:	learn: 1634.2685824	test: 1740.9001535	best: 1740.9001535 (350)	total: 22.1s	remaining: 10m 8s
400:	learn: 1600.3381674	test: 1716.5914387	best: 1716.5914387 (400)	total: 25.2s	remaining: 10m 4s
450:	learn: 1570.5064586	test: 1695.0062682	best: 1695.006

4050:	learn: 1113.0690225	test: 1417.9090131	best: 1417.8998331 (4049)	total: 4m 6s	remaining: 6m 2s
4100:	learn: 1111.1818490	test: 1417.7303097	best: 1417.6815028 (4094)	total: 4m 9s	remaining: 5m 59s
4150:	learn: 1109.0214947	test: 1417.0326319	best: 1417.0326319 (4150)	total: 4m 12s	remaining: 5m 56s
4200:	learn: 1106.5487151	test: 1415.3586643	best: 1415.3586643 (4200)	total: 4m 16s	remaining: 5m 53s
4250:	learn: 1104.5965932	test: 1414.5610779	best: 1414.5610779 (4250)	total: 4m 18s	remaining: 5m 50s
4300:	learn: 1102.8106289	test: 1414.1862291	best: 1414.1862291 (4300)	total: 4m 22s	remaining: 5m 47s
4350:	learn: 1100.7961597	test: 1413.2087152	best: 1413.2087152 (4350)	total: 4m 25s	remaining: 5m 44s
4400:	learn: 1098.7657785	test: 1412.5957992	best: 1412.5725433 (4380)	total: 4m 28s	remaining: 5m 41s
4450:	learn: 1096.7376158	test: 1411.9099024	best: 1411.9099024 (4450)	total: 4m 31s	remaining: 5m 38s
4500:	learn: 1094.7929144	test: 1411.4614609	best: 1411.3797591 (4487)	total

2300:	learn: 1241.0243467	test: 1461.5544176	best: 1461.5544176 (2300)	total: 2m 19s	remaining: 7m 45s
2350:	learn: 1236.1523039	test: 1459.0297727	best: 1459.0146258 (2349)	total: 2m 22s	remaining: 7m 42s
2400:	learn: 1231.7205940	test: 1456.3972632	best: 1456.3972632 (2400)	total: 2m 25s	remaining: 7m 39s
2450:	learn: 1226.6564603	test: 1453.6916199	best: 1453.6916199 (2450)	total: 2m 28s	remaining: 7m 36s
2500:	learn: 1222.4135519	test: 1451.2976366	best: 1451.2976366 (2500)	total: 2m 31s	remaining: 7m 33s
2550:	learn: 1217.9528486	test: 1448.8268476	best: 1448.8268476 (2550)	total: 2m 34s	remaining: 7m 29s
2600:	learn: 1214.1062092	test: 1446.7195854	best: 1446.7195854 (2600)	total: 2m 37s	remaining: 7m 26s
2650:	learn: 1209.9483686	test: 1444.1084958	best: 1444.1084958 (2650)	total: 2m 40s	remaining: 7m 23s
2700:	learn: 1205.8012380	test: 1441.8710310	best: 1441.8710310 (2700)	total: 2m 43s	remaining: 7m 20s
2750:	learn: 1202.3968439	test: 1440.0198899	best: 1440.0198899 (2750)	to

6300:	learn: 1047.3858357	test: 1369.5284268	best: 1369.5284268 (6300)	total: 6m 21s	remaining: 3m 44s
6350:	learn: 1046.2701626	test: 1368.9555967	best: 1368.9555967 (6350)	total: 6m 24s	remaining: 3m 41s
6400:	learn: 1045.1800405	test: 1368.6277187	best: 1368.6249647 (6399)	total: 6m 27s	remaining: 3m 38s
6450:	learn: 1044.0257350	test: 1368.1134874	best: 1368.1134874 (6450)	total: 6m 31s	remaining: 3m 35s
6500:	learn: 1042.8454961	test: 1367.6465329	best: 1367.6384239 (6495)	total: 6m 34s	remaining: 3m 32s
6550:	learn: 1041.6153028	test: 1366.8543014	best: 1366.8299745 (6549)	total: 6m 37s	remaining: 3m 29s
6600:	learn: 1040.2016630	test: 1366.0876209	best: 1366.0876209 (6600)	total: 6m 40s	remaining: 3m 26s
6650:	learn: 1039.0076542	test: 1365.6671783	best: 1365.6598343 (6647)	total: 6m 43s	remaining: 3m 23s
6700:	learn: 1037.7353095	test: 1365.4701149	best: 1365.4087621 (6675)	total: 6m 46s	remaining: 3m 20s
6750:	learn: 1036.5370931	test: 1365.2819255	best: 1365.1772738 (6731)	to

250:	learn: 1733.2994089	test: 1862.2059660	best: 1862.2059660 (250)	total: 15.5s	remaining: 10m 1s
300:	learn: 1683.6776609	test: 1817.8073449	best: 1817.8073449 (300)	total: 18.6s	remaining: 9m 59s
350:	learn: 1639.3862660	test: 1784.4656923	best: 1784.4656923 (350)	total: 21.8s	remaining: 9m 58s
400:	learn: 1607.4676427	test: 1761.2007401	best: 1761.2007401 (400)	total: 24.8s	remaining: 9m 53s
450:	learn: 1582.0713922	test: 1740.7953488	best: 1740.7953488 (450)	total: 27.8s	remaining: 9m 49s
500:	learn: 1553.8186393	test: 1718.7721407	best: 1718.7721407 (500)	total: 30.9s	remaining: 9m 45s
550:	learn: 1528.6024694	test: 1703.8650927	best: 1703.8650927 (550)	total: 33.9s	remaining: 9m 41s
600:	learn: 1507.0016800	test: 1690.6792891	best: 1690.6792891 (600)	total: 36.9s	remaining: 9m 37s
650:	learn: 1485.4285836	test: 1677.4192808	best: 1677.4192808 (650)	total: 39.9s	remaining: 9m 33s
700:	learn: 1470.4880197	test: 1666.0933393	best: 1666.0933393 (700)	total: 42.9s	remaining: 9m 28s


...

8300:	learn: 996.3200776	test: 1404.1773073	best: 1404.1632313 (8299)	total: 8m 26s	remaining: 1m 43s
8350:	learn: 995.4619793	test: 1403.8651883	best: 1403.8651883 (8350)	total: 8m 29s	remaining: 1m 40s
8400:	learn: 994.3402627	test: 1403.4115448	best: 1403.4115448 (8400)	total: 8m 32s	remaining: 1m 37s
8450:	learn: 993.4379476	test: 1403.3036801	best: 1403.3036801 (8450)	total: 8m 35s	remaining: 1m 34s
8500:	learn: 992.4852191	test: 1403.1760785	best: 1403.1168677 (8474)	total: 8m 39s	remaining: 1m 31s
8550:	learn: 991.5725765	test: 1403.0171120	best: 1402.9932441 (8531)	total: 8m 42s	remaining: 1m 28s
8600:	learn: 990.7753721	test: 1403.1918375	best: 1402.9932441 (8531)	total: 8m 45s	remaining: 1m 25s
8650:	learn: 989.6543440	test: 1402.8049017	best: 1402.8049017 (8650)	total: 8m 48s	remaining: 1m 22s
8700:	learn: 988.8550741	test: 1402.6825021	best: 1402.6154883 (8692)	total: 8m 51s	remaining: 1m 19s
8750:	learn: 988.0299473	test: 1402.3792571	best: 1402.3587552 (8749)	total: 8m 54

2250:	learn: 1231.3155276	test: 1435.3933378	best: 1435.3933378 (2250)	total: 2m 16s	remaining: 7m 48s
2300:	learn: 1226.3835855	test: 1434.7183040	best: 1434.6327773 (2299)	total: 2m 19s	remaining: 7m 45s
2350:	learn: 1221.4293820	test: 1432.4271364	best: 1432.4271364 (2350)	total: 2m 22s	remaining: 7m 42s
2400:	learn: 1217.1672722	test: 1430.8564434	best: 1430.8564434 (2400)	total: 2m 25s	remaining: 7m 39s
2450:	learn: 1213.4191645	test: 1429.8192598	best: 1429.8192598 (2450)	total: 2m 28s	remaining: 7m 36s
2500:	learn: 1210.3385941	test: 1428.3057886	best: 1428.3057886 (2500)	total: 2m 31s	remaining: 7m 33s
2550:	learn: 1206.2245879	test: 1427.5869970	best: 1427.3464817 (2535)	total: 2m 34s	remaining: 7m 30s
2600:	learn: 1202.4558253	test: 1426.3018011	best: 1426.0348170 (2592)	total: 2m 37s	remaining: 7m 27s
2650:	learn: 1199.2188892	test: 1425.5455245	best: 1425.5455245 (2650)	total: 2m 40s	remaining: 7m 24s
2700:	learn: 1195.5458279	test: 1424.6706733	best: 1424.5608197 (2696)	to

6250:	learn: 1051.2903090	test: 1387.6853374	best: 1387.3605194 (6207)	total: 6m 20s	remaining: 3m 48s
6300:	learn: 1049.9898889	test: 1388.2596975	best: 1387.3605194 (6207)	total: 6m 23s	remaining: 3m 45s
6350:	learn: 1048.8145460	test: 1388.2982534	best: 1387.3605194 (6207)	total: 6m 26s	remaining: 3m 42s
6400:	learn: 1047.5226938	test: 1388.3242633	best: 1387.3605194 (6207)	total: 6m 29s	remaining: 3m 39s
bestTest = 1387.360519
bestIteration = 6207
Shrink model to first 6208 iterations.
-------------- Fold 3 ending ---------------
-------------- Fold 4 starting ---------------
0:	learn: 3799.2363412	test: 3727.3403793	best: 3727.3403793 (0)	total: 98.9ms	remaining: 16m 28s
50:	learn: 2644.9297206	test: 2584.6559296	best: 2584.6559296 (50)	total: 3.01s	remaining: 9m 47s
100:	learn: 2170.4779418	test: 2127.6134841	best: 2127.6134841 (100)	total: 5.95s	remaining: 9m 43s
150:	learn: 1949.8036921	test: 1920.5593675	best: 1920.5593675 (150)	total: 9.12s	remaining: 9m 54s
200:	learn: 1820.

3750:	learn: 1134.3014760	test: 1431.7068714	best: 1431.7068714 (3750)	total: 3m 46s	remaining: 6m 17s
3800:	learn: 1132.1723348	test: 1430.8818961	best: 1430.8818961 (3800)	total: 3m 49s	remaining: 6m 14s
3850:	learn: 1129.3654824	test: 1429.9817978	best: 1429.7669860 (3847)	total: 3m 52s	remaining: 6m 11s
3900:	learn: 1127.1284766	test: 1429.3225213	best: 1429.3225213 (3898)	total: 3m 55s	remaining: 6m 8s
3950:	learn: 1124.3870314	test: 1428.5549210	best: 1428.5549210 (3950)	total: 3m 58s	remaining: 6m 5s
4000:	learn: 1122.2631687	test: 1427.6349327	best: 1427.6349327 (4000)	total: 4m 1s	remaining: 6m 2s
4050:	learn: 1120.2222318	test: 1426.7893024	best: 1426.7893024 (4050)	total: 4m 4s	remaining: 5m 59s
4100:	learn: 1118.2188562	test: 1426.1370640	best: 1426.1021800 (4097)	total: 4m 7s	remaining: 5m 56s
4150:	learn: 1115.5911662	test: 1425.4150576	best: 1425.3966976 (4149)	total: 4m 10s	remaining: 5m 53s
4200:	learn: 1113.3196590	test: 1424.3971494	best: 1424.3971494 (4200)	total: 4

7750:	learn: 1011.8193267	test: 1395.2436356	best: 1395.1873316 (7746)	total: 7m 51s	remaining: 2m 16s
7800:	learn: 1010.8141199	test: 1395.3115675	best: 1395.1392897 (7764)	total: 7m 54s	remaining: 2m 13s
7850:	learn: 1009.7036495	test: 1395.5806943	best: 1395.1392897 (7764)	total: 7m 57s	remaining: 2m 10s
7900:	learn: 1008.7256767	test: 1395.2517446	best: 1395.1278147 (7885)	total: 8m	remaining: 2m 7s
7950:	learn: 1007.7425018	test: 1394.9133088	best: 1394.9111668 (7947)	total: 8m 3s	remaining: 2m 4s
8000:	learn: 1006.8049208	test: 1394.8781189	best: 1394.8770479 (7967)	total: 8m 6s	remaining: 2m 1s
8050:	learn: 1005.9351186	test: 1394.8949489	best: 1394.8352789 (8040)	total: 8m 9s	remaining: 1m 58s
8100:	learn: 1004.9331249	test: 1394.8216619	best: 1394.7552600 (8094)	total: 8m 13s	remaining: 1m 55s
8150:	learn: 1003.9595890	test: 1394.6734050	best: 1394.6371441 (8134)	total: 8m 16s	remaining: 1m 52s
8200:	learn: 1003.1437191	test: 1394.6769240	best: 1394.6371441 (8134)	total: 8m 19

In [18]:
count = 0
models_list = []
for train_idx, val_idx in gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']):
    print('-------------- Fold {} starting ---------------'.format(count))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]
    
    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]
    
    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)
    
    model = catboost.CatBoostRegressor(iterations=10000, depth=6, task_type='GPU', loss_function='Quantile:alpha=0.875')
    
    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)
    
    model.save_model('./Training/Training6_20220207/catboost_quantile0875_{}'.format(count))
    
    print('-------------- Fold {} ending ---------------'.format(count))
    count += 1

-------------- Fold 0 starting ---------------
0:	learn: 9303.2721953	test: 9465.0984982	best: 9465.0984982 (0)	total: 93.4ms	remaining: 15m 33s
50:	learn: 3586.5229634	test: 3914.0275877	best: 3914.0275877 (50)	total: 3.14s	remaining: 10m 13s
100:	learn: 2606.2514728	test: 2948.8880283	best: 2948.8880283 (100)	total: 6.05s	remaining: 9m 52s
150:	learn: 2325.2961672	test: 2677.3793862	best: 2677.3793862 (150)	total: 9.24s	remaining: 10m 2s
200:	learn: 2197.2602888	test: 2548.6942544	best: 2548.6942544 (200)	total: 12.4s	remaining: 10m 4s
250:	learn: 2103.9644993	test: 2470.6952584	best: 2470.6952584 (250)	total: 15.5s	remaining: 10m 1s
300:	learn: 2037.7138370	test: 2424.4742602	best: 2424.4742602 (300)	total: 18.6s	remaining: 9m 58s
350:	learn: 1980.4653650	test: 2380.4911858	best: 2380.4911858 (350)	total: 21.5s	remaining: 9m 52s
400:	learn: 1931.5555242	test: 2338.1822320	best: 2338.1822320 (400)	total: 24.6s	remaining: 9m 47s
450:	learn: 1888.5305070	test: 2309.2208022	best: 2309.2

750:	learn: 1716.1095979	test: 1962.7942683	best: 1962.7942683 (750)	total: 45.6s	remaining: 9m 21s
800:	learn: 1693.9598460	test: 1954.2382298	best: 1954.2382298 (800)	total: 48.6s	remaining: 9m 18s
850:	learn: 1669.8412952	test: 1943.1560165	best: 1943.1275586 (849)	total: 51.6s	remaining: 9m 15s
900:	learn: 1647.4195100	test: 1936.0121635	best: 1935.8392740 (896)	total: 54.7s	remaining: 9m 11s
950:	learn: 1628.4373087	test: 1930.9013679	best: 1930.7388824 (948)	total: 57.7s	remaining: 9m 8s
1000:	learn: 1612.3292049	test: 1927.7142352	best: 1927.7142352 (1000)	total: 1m	remaining: 9m 5s
1050:	learn: 1596.2304341	test: 1922.3796013	best: 1921.9456947 (1044)	total: 1m 3s	remaining: 9m 2s
1100:	learn: 1579.7107614	test: 1921.7316484	best: 1921.1043504 (1089)	total: 1m 6s	remaining: 8m 59s
1150:	learn: 1565.6522403	test: 1918.7906919	best: 1918.7906919 (1150)	total: 1m 9s	remaining: 8m 56s
1200:	learn: 1554.4026388	test: 1913.1538840	best: 1913.1538840 (1200)	total: 1m 12s	remaining: 8m

750:	learn: 1707.9798112	test: 1954.4180042	best: 1954.4180042 (750)	total: 45.3s	remaining: 9m 17s
800:	learn: 1688.0171885	test: 1940.9508346	best: 1940.9508346 (800)	total: 48.3s	remaining: 9m 14s
850:	learn: 1669.7657134	test: 1925.3155089	best: 1925.3155089 (850)	total: 51.3s	remaining: 9m 11s
900:	learn: 1651.8301824	test: 1912.8182031	best: 1912.8182031 (900)	total: 54.3s	remaining: 9m 8s
950:	learn: 1633.1627013	test: 1902.2783730	best: 1902.2644501 (949)	total: 57.3s	remaining: 9m 5s
1000:	learn: 1616.4961490	test: 1892.2313544	best: 1892.2313544 (1000)	total: 1m	remaining: 9m 3s
1050:	learn: 1598.8022044	test: 1881.1111972	best: 1881.1111972 (1050)	total: 1m 3s	remaining: 9m
1100:	learn: 1585.5288772	test: 1872.9015496	best: 1872.8238258 (1099)	total: 1m 6s	remaining: 8m 57s
1150:	learn: 1570.6511962	test: 1863.6761383	best: 1863.6761383 (1150)	total: 1m 9s	remaining: 8m 55s
1200:	learn: 1557.0709518	test: 1855.5047406	best: 1855.5047406 (1200)	total: 1m 12s	remaining: 8m 52s

350:	learn: 1996.6561193	test: 2245.3409196	best: 2245.3409196 (350)	total: 21.2s	remaining: 9m 41s
400:	learn: 1946.2459945	test: 2212.9256014	best: 2212.9256014 (400)	total: 24.1s	remaining: 9m 38s
450:	learn: 1903.7465563	test: 2182.0338560	best: 2182.0338560 (450)	total: 27.1s	remaining: 9m 33s
500:	learn: 1867.6039416	test: 2161.4933170	best: 2161.4933170 (500)	total: 30.1s	remaining: 9m 29s
550:	learn: 1830.3915544	test: 2140.8842341	best: 2140.8753601 (546)	total: 33.1s	remaining: 9m 26s
600:	learn: 1799.3802372	test: 2128.3387035	best: 2128.3387035 (600)	total: 36.1s	remaining: 9m 23s
650:	learn: 1772.5384626	test: 2115.3470252	best: 2115.2677713 (648)	total: 39.1s	remaining: 9m 21s
700:	learn: 1744.0218024	test: 2104.2152134	best: 2103.2748762 (695)	total: 42s	remaining: 9m 17s
750:	learn: 1720.9615637	test: 2096.8517885	best: 2094.3635516 (746)	total: 45s	remaining: 9m 14s
800:	learn: 1699.4157140	test: 2093.5558653	best: 2093.5558653 (800)	total: 48s	remaining: 9m 11s
850:	l